## Step 1. Making Preparations

### Importing libraries

In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


### Initializing Random Number

In [2]:
seed = 7
numpy.random.seed(seed)

### Loading Data

In [3]:
# load dataset
dataframe = pandas.read_csv('C:/Users/FC/Documents/Deep_Learning_Project_Two/iris.csv', header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

### Label Encoding

In [4]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

## Step 3: Define the Neural Network Baseline Model

In [5]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)


## Step 4. Evaluate The Model with k-Fold Cross Validation

In [8]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 97.33% (4.42%)


## Step 5. Tuning Layers and Number of Neurons in The Model

### Step 5.2. Evaluate a Larger Network

In [5]:
# define Larger model
def Larger_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
estimator = KerasClassifier(build_fn=Larger_model, epochs=200, batch_size=5, verbose=0)

In [8]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 98.67% (2.67%)


## Step 6. Really Scaling up: developing a model that overfits

In [9]:
# define overfit model
def overfit_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
estimator = KerasClassifier(build_fn=overfit_model, epochs=200, batch_size=5, verbose=0)

In [15]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("overfit: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 88.67% (21.30%)


## Step 7. Tuning the Model

In [5]:
# define baseline model
def tuned_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
estimator = KerasClassifier(build_fn=tuned_model, epochs=200, batch_size=5, verbose=0)

In [8]:
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Tuned_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Tuned_model: 98.67% (2.67%)


## Step 8. Rewriting the code using the Keras Functional API

In [6]:
import keras
from keras import layers
def functional_model():
    # create model, write code below
    inputs = keras.Input(shape=(4,))
    x = layers.Dense(8,activation='relu')(inputs)
    outputs = layers.Dense(3,activation='softmax')(x)
    model = keras.Model(inputs,outputs)
    # Compile model
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=functional_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("funtional_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

funtional_model: 98.22% (2.95%)


 ## Step 9. Rewriting the code by doing Model Subclassing

In [23]:
import keras
from keras import layers
class Mymodel(keras.Model):
    def __init__(self):
        super(Mymodel,self).__init__()
        self.dense1 = Dense(8, activation = 'relu')
        self.dense2 = Dense(3, activation = 'softmax')
    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)
def subclass_model():
    inputs = keras.Input(shape=(4,))
    model = Mymodel()
    outputs = model.call(inputs)
    model = keras.Model(inputs, outputs)
    # Compile model, write code below
    model.compile(optimizer='adam',loss='categorical_crossentropy' ,metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn= subclass_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("funtional_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [24]:
estimator = KerasClassifier(build_fn= subclass_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("funtional_model: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

funtional_model: 98.22% (2.18%)


## Step 10. Rewriting the code without using scikit-learn

In [39]:
data = pandas.read_csv('C:/Users/FC/Documents/Deep_Learning_Project_Two/iris.csv', header=None)

In [41]:
data[4][data[4]=='Iris-setosa']=0
data[4][data[4]=='Iris-versicolor']=1
data[4][data[4]=='Iris-virginica']=2

C:\Users\FC\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\FC\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\FC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
data_values = data.values
numpy.random.seed(7)
numpy.random.shuffle(data_values)

In [66]:
train_data = data_values[:120,0:4].astype(float)
train_labels =data_values[:120,4]
test_data= data_values[120:,:4].astype(float)
test_labels= data_values[120:,4]

In [67]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [76]:
from keras import models
from keras import layers
def keras_model():
    # create model, write code below
    model = models.Sequential()
    model.add(layers.Dense(8, activation='relu', input_shape=(4,)))
    model.add(layers.Dense(3,activation='softmax'))
    # Compile model, write code below
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [77]:
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 200
all_scores = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_labels[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([train_labels[:i * num_val_samples],train_labels[(i + 1) * num_val_samples:]],axis=0)
    model = keras_model()
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [80]:
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('test_acc:', test_acc)

30/30 [==============================] - 0s 0us/step
test_acc: 0.9333333373069763
